# 1. Imports

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras import layers, models #type: ignore
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# 2. Load Data

In [ ]:
df = pd.read_csv("../data/raw/merged-stock-data.csv")
print("Available columns:", df.columns.tolist())
print(f"Initial data shape: {df.shape}")

def add_advanced_technical_features(df):
    """Add advanced technical indicators used by professional traders"""
    df = df.copy()
    
    # Basic technical features
    df['ma_5'] = df['close'].rolling(5).mean()
    df['ma_20'] = df['close'].rolling(20).mean()
    df['price_change'] = df['close'].pct_change()
    df['volatility'] = df['close'].rolling(10).std()
    df['hl_spread'] = (df['high'] - df['low']) / df['close']
    df['oc_spread'] = (df['close'] - df['open']) / df['open']
    
    # Advanced momentum indicators
    df['rsi'] = calculate_rsi(df['close'], 14)
    df['macd'], df['macd_signal'] = calculate_macd(df['close'])
    df['bb_upper'], df['bb_lower'], df['bb_width'] = calculate_bollinger_bands(df['close'])
    
    # Volume-based indicators (if volume available)
    if 'Volume' in df.columns:
        df['volume_sma'] = df['Volume'].rolling(20).mean()
        df['volume_ratio'] = df['Volume'] / df['volume_sma']
        df['vwap'] = calculate_vwap(df)
    
    # Support/Resistance levels
    df['support'], df['resistance'] = calculate_support_resistance(df)
    
    # Market structure
    df['higher_high'] = calculate_higher_highs(df['high'])
    df['lower_low'] = calculate_lower_lows(df['low'])
    
    return df.dropna()

def calculate_rsi(prices, period=14):
    """Calculate Relative Strength Index"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """Calculate MACD indicator"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    return macd, macd_signal

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    """Calculate Bollinger Bands"""
    sma = prices.rolling(period).mean()
    std = prices.rolling(period).std()
    upper = sma + (std * std_dev)
    lower = sma - (std * std_dev)
    width = (upper - lower) / sma
    return upper, lower, width

def calculate_vwap(df):
    """Calculate Volume Weighted Average Price"""
    if 'Volume' in df.columns:
        return (df['close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return pd.Series(index=df.index, dtype=float)

def calculate_support_resistance(df, window=20):
    """Calculate support and resistance levels"""
    support = df['low'].rolling(window).min()
    resistance = df['high'].rolling(window).max()
    return support, resistance

def calculate_higher_highs(highs, period=5):
    """Detect higher highs pattern"""
    return highs > highs.shift(1).rolling(period).max()

def calculate_lower_lows(lows, period=5):
    """Detect lower lows pattern"""
    return lows < lows.shift(1).rolling(period).min()

print("Adding advanced technical features...")
df = add_advanced_technical_features(df)

feature_cols = [
    "open", "high", "low", "close",
    "ma_5", "ma_20", "price_change", "volatility", "hl_spread", "oc_spread"
]

print(f"Enhanced features: {len(feature_cols)} features")
print(f"Data shape after feature engineering: {df.shape}")

x_raw = df[feature_cols].values
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x_raw)
print("Features scaled successfully")

# 3. Load Saved Pattern Recognition Model

In [ ]:
pattern_model = load_model("../models/candlestick_cnn_lstm.h5")

lookback = 20

x_seq = []
for i in range(lookback, len(x_scaled)):
    x_seq.append(x_scaled[i-lookback:i])

x_seq = np.array(x_seq)

patterns_preds = np.argmax(pattern_model.predict(x_seq), axis=1)
print("predicted pattern shape : ", patterns_preds.shape)

# 4. Create Trading Labels

In [ ]:
def generate_advanced_trading_signals(df, lookback):
    """Generate sophisticated trading signals using multiple criteria"""
    
    # Multi-timeframe analysis - ensure we're not using future data
    returns_1d = df["close"].pct_change().values
    returns_5d = df["close"].pct_change(periods=5).values
    returns_20d = df["close"].pct_change(periods=20).values
    
    # Trim arrays to match lookback
    returns_1d = returns_1d[lookback:]
    returns_5d = returns_5d[lookback:]
    returns_20d = returns_20d[lookback:]
    
    # Use more conservative thresholds to reduce noise
    vol_threshold = np.percentile(df['volatility'].values[lookback:], 75)  # More conservative
    volatility = df['volatility'].values[lookback:]
    vol_regime = volatility > vol_threshold
    
    # More conservative signal criteria
    y_trading = np.ones(len(returns_1d), dtype=int)  # Default Hold
    
    # Strong Buy signals - more restrictive
    strong_buy = (
        (returns_1d > np.percentile(returns_1d, 85)) &  # Very strong short-term momentum
        (returns_5d > np.percentile(returns_5d, 70)) &  # Strong medium-term trend
        (returns_20d > np.percentile(returns_20d, 60)) &  # Positive long-term trend
        (~vol_regime)  # Low volatility environment
    )
    
    # Strong Sell signals - more restrictive
    strong_sell = (
        (returns_1d < np.percentile(returns_1d, 15)) &  # Very strong negative momentum
        (returns_5d < np.percentile(returns_5d, 30)) &  # Strong negative medium-term trend
        (returns_20d < np.percentile(returns_20d, 40)) &  # Negative long-term trend
        (~vol_regime)  # Low volatility environment
    )
    
    # Apply signals - be more conservative
    y_trading[strong_buy] = 2  # Buy
    y_trading[strong_sell] = 0  # Sell
    
    print("Advanced Trading Signal Distribution:")
    print(f"Sell (0): {np.sum(y_trading == 0)} ({np.sum(y_trading == 0)/len(y_trading)*100:.1f}%)")
    print(f"Hold (1): {np.sum(y_trading == 1)} ({np.sum(y_trading == 1)/len(y_trading)*100:.1f}%)")
    print(f"Buy (2): {np.sum(y_trading == 2)} ({np.sum(y_trading == 2)/len(y_trading)*100:.1f}%)")
    
    print(f"\nMarket Regime Analysis:")
    print(f"High volatility periods: {np.sum(vol_regime)/len(vol_regime)*100:.1f}%")
    print(f"Strong buy signals: {np.sum(strong_buy)} ({np.sum(strong_buy)/len(y_trading)*100:.1f}%)")
    print(f"Strong sell signals: {np.sum(strong_sell)} ({np.sum(strong_sell)/len(y_trading)*100:.1f}%)")
    
    return y_trading

# 5. Combine Features

In [ ]:
# Use pattern probabilities instead of one-hot for richer information
pattern_probs = pattern_model.predict(x_seq)  # Soft probabilities
print("Pattern probabilities shape:", pattern_probs.shape)

# Add more technical indicators as per-timestep features
def add_more_indicators(df, lookback):
    """Add additional technical indicators"""
    df = df.copy()
    
    # EMAs
    df['ema_12'] = df['close'].ewm(span=12).mean()
    df['ema_26'] = df['close'].ewm(span=26).mean()
    
    # Additional features if not already present
    if 'rsi' not in df.columns:
        df['rsi'] = calculate_rsi(df['close'], 14)
    if 'macd' not in df.columns:
        df['macd'], df['macd_signal'] = calculate_macd(df['close'])
    
    # Stochastic oscillator
    df['stoch_k'] = ((df['close'] - df['low'].rolling(14).min()) / 
                     (df['high'].rolling(14).max() - df['low'].rolling(14).min())) * 100
    df['stoch_d'] = df['stoch_k'].rolling(3).mean()
    
    return df.dropna()

# Add enhanced features to the same dataframe to maintain alignment
df_enhanced = add_more_indicators(df, lookback)

# Enhanced feature set
enhanced_feature_cols = [
    "open", "high", "low", "close",
    "ma_5", "ma_20", "ema_12", "ema_26", 
    "price_change", "volatility", "hl_spread", "oc_spread",
    "rsi", "macd", "macd_signal", "stoch_k", "stoch_d"
]

# Ensure same length by aligning with pattern predictions
min_length = min(len(df_enhanced) - lookback, len(pattern_probs))
print(f"Aligning to minimum length: {min_length}")

# Generate trading signals first (now that min_length is defined)
y_trading = generate_advanced_trading_signals(df_enhanced, lookback)
y_trading_aligned = y_trading[:min_length]
print(f"Aligned trading signals shape: {y_trading_aligned.shape}")

# Get enhanced features and scale them
x_enhanced = df_enhanced[enhanced_feature_cols].values
scaler_enhanced = MinMaxScaler()
x_enhanced_scaled = scaler_enhanced.fit_transform(x_enhanced)

# Create sequences for CNN+LSTM (keep 3D structure) - align with pattern predictions
x_seq_enhanced = []
for i in range(lookback, lookback + min_length):
    x_seq_enhanced.append(x_enhanced_scaled[i-lookback:i])

x_seq_enhanced = np.array(x_seq_enhanced)
print("Enhanced sequence shape:", x_seq_enhanced.shape)

# Trim pattern probabilities to match
pattern_probs_aligned = pattern_probs[:min_length]
print("Aligned pattern probabilities shape:", pattern_probs_aligned.shape)

# Combine sequences with pattern probabilities as additional features per timestep
sequence_length, n_base_features = x_seq_enhanced.shape[1], x_seq_enhanced.shape[2]
n_pattern_features = pattern_probs_aligned.shape[1]

# Create final trading sequences
x_trading_seq = np.zeros((min_length, sequence_length, n_base_features + n_pattern_features))
x_trading_seq[:, :, :n_base_features] = x_seq_enhanced

# Add pattern probabilities as additional features at each timestep
for i in range(sequence_length):
    x_trading_seq[:, i, n_base_features:] = pattern_probs_aligned

print("Final trading sequence shape:", x_trading_seq.shape)

# 6. Train/Test Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_trading_seq, y_trading_aligned, test_size=0.2, shuffle=False)

print("Training set shape:", x_train.shape)
print("Test set shape:", x_test.shape)
print("Training labels distribution:", np.bincount(y_train))
print("Test labels distribution:", np.bincount(y_test))

# 7. Build Trading Signal Model (CNN + LSTM)

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

# Hyperparameters for tuning
sequence_length = 20  # Can tune from 5-30
kernel_size = 5      # Can tune from 3-7
lstm_units = 64      # Can tune from 32-128

input_shape = x_train.shape[1:]  # (sequence_length, n_features)
num_classes = 3

# CNN + LSTM Model for Time Series - Simplified to reduce overfitting
inputs = layers.Input(shape=input_shape)

# Simplified CNN layers - reduce complexity
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv1D(filters=64, kernel_size=kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Dropout(0.4)(x)

# Simplified LSTM layers
x = layers.LSTM(lstm_units//2, return_sequences=False, dropout=0.4, recurrent_dropout=0.4)(x)

# Simplified dense layers
x = layers.Dense(32, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(num_classes, activation='softmax')(x)
trading_model = models.Model(inputs, outputs)

# Lower learning rate for stability
initial_learning_rate = 0.0005

trading_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

trading_model.summary()

# 8. Train Model

In [ ]:
# Compute class weights to handle imbalance - but cap them to reduce instability
class_weights = compute_class_weight('balanced', 
                                   classes=np.unique(y_train), 
                                   y=y_train)

# Cap class weights to prevent extreme values
max_weight = 3.0
class_weights = np.clip(class_weights, 0.5, max_weight)
class_weight_dict = dict(enumerate(class_weights))
print("Capped class weights:", class_weight_dict)

callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True, monitor='val_loss', min_delta=0.001),
    ReduceLROnPlateau(patience=8, factor=0.5, monitor='val_loss', verbose=1, min_lr=1e-6)
]

history = trading_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,  # Smaller batch size for more stable gradients
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

# 9. Evaluate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Basic evaluation
test_loss, test_accuracy = trading_model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# Detailed predictions
y_pred = trading_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, 
                          target_names=['Sell', 'Hold', 'Buy']))

# Confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Sell', 'Hold', 'Buy'],
            yticklabels=['Sell', 'Hold', 'Buy'])
plt.title('Trading Signal Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# 10. Plot Training History

In [ ]:
plt.figure(figsize=(12, 4))

# Plot training history
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Model Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Model Loss")

plt.tight_layout()
plt.show()

# 11. Walk-Forward Validation (Optional - Better than Single Split)

In [ ]:
def walk_forward_validation(x_data, y_data, model_builder, n_splits=5):
    """
    Perform walk-forward validation for time series data
    Better than random split as it respects temporal order
    """
    total_samples = len(x_data)
    split_size = total_samples // n_splits
    
    accuracies = []
    
    for i in range(n_splits - 1):
        # Progressive training set (expanding window)
        train_end = (i + 2) * split_size
        test_start = train_end
        test_end = test_start + split_size
        
        # Split data
        x_train_fold = x_data[:train_end]
        y_train_fold = y_data[:train_end]
        x_test_fold = x_data[test_start:test_end]
        y_test_fold = y_data[test_start:test_end]
        
        print(f"\nFold {i+1}:")
        print(f"Train: 0 to {train_end} ({len(x_train_fold)} samples)")
        print(f"Test: {test_start} to {test_end} ({len(x_test_fold)} samples)")
        
        # Build and train model
        model = model_builder(x_train_fold.shape[1:])
        
        # Class weights for this fold
        fold_class_weights = compute_class_weight('balanced', 
                                                classes=np.unique(y_train_fold), 
                                                y=y_train_fold)
        fold_class_weight_dict = dict(enumerate(fold_class_weights))
        
        # Train model
        model.fit(x_train_fold, y_train_fold,
                 validation_split=0.2,
                 epochs=30,  # Reduced for faster validation
                 batch_size=64,
                 class_weight=fold_class_weight_dict,
                 verbose=0)
        
        # Evaluate
        _, accuracy = model.evaluate(x_test_fold, y_test_fold, verbose=0)
        accuracies.append(accuracy)
        print(f"Fold {i+1} accuracy: {accuracy:.4f}")
    
    return accuracies

def create_cnn_lstm_model(input_shape, 
                         sequence_length=20, 
                         kernel_size=5, 
                         lstm_units=64):
    """Model builder function for walk-forward validation"""
    inputs = layers.Input(shape=input_shape)
    
    # Multi-scale CNN
    conv1 = layers.Conv1D(32, 3, activation='relu', padding='same')(inputs)
    conv2 = layers.Conv1D(32, kernel_size, activation='relu', padding='same')(inputs)
    conv3 = layers.Conv1D(32, 7, activation='relu', padding='same')(inputs)
    
    x = layers.Concatenate()([conv1, conv2, conv3])
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Conv1D(64, kernel_size, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Dropout(0.3)(x)
    
    # LSTM layers
    x = layers.LSTM(lstm_units, return_sequences=True, dropout=0.3)(x)
    x = layers.LSTM(lstm_units//2, dropout=0.3)(x)
    
    # Dense layers
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    outputs = layers.Dense(3, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model

# Run walk-forward validation (uncomment to run)
# print("Running walk-forward validation...")
# wf_accuracies = walk_forward_validation(x_trading_seq, y_trading_aligned, create_cnn_lstm_model)
# print(f"\nWalk-forward validation results:")
# print(f"Mean accuracy: {np.mean(wf_accuracies):.4f} ± {np.std(wf_accuracies):.4f}")
# print(f"Individual fold accuracies: {wf_accuracies}")

print("Walk-forward validation function defined. Uncomment to run.")